# Athlete Dashboard - Preparation
Development sandbox for the athlete dashboard user interface.

In [1]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
%matplotlib inline

import seaborn as sns
import panel as pn
import panel.widgets as pnw
import panel.pane as pnp
import param
import hvplot as hv

In [2]:
full_rounds = pd.read_csv('../data/scraped/cleaned/rounds_splits.csv')

# replace 0 positions and laptimes with NaN
pos_cols = [f'lap_{x}_position' for x in range(1, 46)]
laptime_cols = [f'lap_{x}_laptime' for x in range(1, 46)]
full_rounds[pos_cols] = full_rounds[pos_cols].replace(0.0, np.nan)
full_rounds[laptime_cols] = full_rounds[laptime_cols].replace(0.0, np.nan)

C:\Users\ahale\Anaconda3\envs\shorttrack\lib\site-packages\IPython\core\interactiveshell.py:3071: DtypeWarning: Columns (15,19,20,22,23,25,26,28,29,31,34,37,40,43,46,47,49,50,52,53,55,56,58,59,62,65,68,71,74,77,80,83,86,89,92,95,98,101,104,107,109,110,113,116,119,122,125,128,131,134,137,140,143,146,149,151,152) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [3]:
individual_events = full_rounds[full_rounds['event'].isin({'500m', '1000m', '1500m'})]

In [39]:
ui_template = pn.template.MaterialTemplate(title='Short Track Athlete Profile')
pn.config.sizing_mode = 'stretch_width'

# widgets containing selectors for the user to refine which data they are seeing
athlete_name = pn.widgets.Select(name='Athlete',
                                 options=list(individual_events['Name'].unique()))
event = pn.widgets.RadioButtonGroup(name='Event', 
                                    options=list(individual_events['event'].unique()) + ['All'])
start_position = pn.widgets.RadioButtonGroup(name='Start Position', 
                                             options=[i for i in range(1, 9)])
position_gain_loss = pn.widgets.RadioButtonGroup(name='Position Gain/Loss',
                                                 options=[-4, -3, -2, -1, 1, 2, 3, 4])

def get_athlete_races(athlete_name, event):
    athlete_races = individual_events[individual_events['Name'] == athlete_name]
    if event == 'All':
        return athlete_races
    else:
        return athlete_races[athlete_races['event'] == event]

def get_laptimes(athlete_races):
    race_details_cols = list(athlete_races.columns[:16]) + ['instance_of_event_in_competition']
    lap_details_cols = race_details_cols.copy()
    lap_details_cols.extend(['lap', 'laptime', 'lap_start_position', 'lap_end_position', 'position_change'])
    laptimes = pd.DataFrame(columns=lap_details_cols)

    for index, athlete_race in athlete_races.iterrows():
        start_lap = 2 if athlete_race['event'] in ['500m', '1500m'] else 1
        for i in range(start_lap, 46):
            try:
                laptime = float(athlete_race[f'lap_{i}_laptime'])
            except Exception:
                laptime = np.nan

            # TODO use standard deviation to filter out erroneous laptimes instead of the 7.8 threshold
            if not np.isnan(laptime) and laptime > 7.8:
                lap_details = athlete_race[race_details_cols]
                lap_details['lap'] = i
                lap_details['laptime'] = laptime
                lap_details['lap_start_position'] = float(athlete_race[f'lap_{i - 1}_position']) if i > 1 else float(athlete_race['Start Pos.'])
                lap_details['lap_end_position'] = float(athlete_race[f'lap_{i}_position'])
                lap_details['position_change'] = (-1) * (lap_details['lap_end_position'] - lap_details['lap_start_position'])

                laptimes = laptimes.append(lap_details)
    laptimes['lap'] = laptimes['lap'].astype('int')
    return laptimes

@pn.depends(athlete_name=athlete_name, event=event)
def first_lap_positions(athlete_name, event):
    plot = plt.figure()
    plot.suptitle('Most Common Position to Start the Race')
    plot.add_subplot(111).hist(get_athlete_races(athlete_name, event)['lap_1_position'])
    plt.xlabel('Position')
    plt.ylabel('Frequency')
    plt.close(plot)
    return plot

@pn.depends(athlete_name=athlete_name)
def half_lap_500m(athlete_name):
    mean_start_time = round(get_athlete_races(athlete_name, event='500m')['lap_1_laptime'].astype('float').mean(), 3)
    return pn.indicators.Number(name='Mean 500m Half-Lap Start Time', value=mean_start_time, format='{value}s')

@pn.depends(athlete_name=athlete_name)
def half_lap_500m_hist(athlete_name):
    start_times = get_athlete_races(athlete_name, event='500m')['lap_1_laptime'].astype('float')
    plot = plt.figure()
    plot.suptitle('500m Half-Lap Start Time')
    plot.add_subplot(111).hist(start_times[start_times < 9])
    plt.xlabel('Half-Lap Start Time')
    plt.ylabel('Frequency')
    plt.close(plot)
    return plot

@pn.depends(athlete_name=athlete_name, start_position=start_position)
def start_performance_500m(athlete_name, start_position):
    all_500m_races = get_athlete_races(athlete_name, event='500m')
    start_performances = all_500m_races[all_500m_races['Start Pos.'] == int(start_position)]['lap_1_position'].astype('float')

    plot = plt.figure()
    plot.suptitle(f'500m Start Performance from Lane {start_position}')
    plot.add_subplot(111).hist(start_performances)
    plt.xlabel('Position after Start')
    plt.ylabel('Frequency')
    plt.close(plot)
    return plot

@pn.depends(athlete_name=athlete_name)
def fastest_leading_laptimes(athlete_name):
    laptimes = get_laptimes(get_athlete_races(athlete_name=athlete_name, event='All'))
    return pn.indicators.Number(name='Fastest Leading Laptimes', 
                                value=round(laptimes[laptimes['lap_end_position'] == 1]['laptime'].nsmallest(25).mean(), 3), 
                                format='{value}s')

@pn.depends(athlete_name=athlete_name)
def fastest_following_laptimes(athlete_name):
    laptimes = get_laptimes(get_athlete_races(athlete_name=athlete_name, event='All'))
    return pn.indicators.Number(name='Fastest Leading Laptimes', 
                                value=round(laptimes[laptimes['lap_end_position'] != 1]['laptime'].nsmallest(25).mean(), 3), 
                                format='{value}s')

@pn.depends(athlete_name=athlete_name, event=event, position_gain_loss=position_gain_loss)
def likely_lap_to_pass(athlete_name, event, position_gain_loss):
    laptimes = get_laptimes(get_athlete_races(athlete_name=athlete_name, event=event))
    
    plot = plt.figure()
    plot.suptitle('Most Common Lap to Pass')
    plot.add_subplot(111).hist(laptimes[laptimes['position_change'] == position_gain_loss]['lap'])
    plt.xlabel('Lap')
    plt.ylabel('Frequency')
    plt.close(plot)
    return plot

@pn.depends(athlete_name=athlete_name, event=event)
def x_plus_y_position_selection(athlete_name, event):
    athlete_races = get_athlete_races(athlete_name=athlete_name, event=event)
    advancing_races = athlete_races[athlete_races['Qual.'].isin(['Q', 'q', 'QA', 'qA'])][['Place']]
    advancing_races['Place'] = advancing_races['Place'].astype('int')
    
    plot = plt.figure()
    plot.suptitle('X + Y Position Selection')
    plot.add_subplot(111).hist(advancing_races['Place'])
    plt.xlabel('Place')
    plt.ylabel('Frequency')
    plt.close(plot)
    return plot

@pn.depends(athlete_name=athlete_name)
def leading_pace_1500m(athlete_name):
    laptimes = get_laptimes(get_athlete_races(athlete_name=athlete_name, event='1500m'))
    leading_pace = laptimes[(laptimes['lap'] > 1) & 
                            (laptimes['lap'] < 5) & 
                            (laptimes['lap_start_position'] == 1) &
                            (laptimes['position_change'] == 0)]['laptime'].mean()
    
    return pn.indicators.Number(name='1500m Leading Pace', 
                                value=round(leading_pace, 2), 
                                format='{value}s')

@pn.depends(athlete_name=athlete_name)
def pace_instigation_1500m(athlete_name=athlete_name):
    laptimes = get_laptimes(get_athlete_races(athlete_name=athlete_name, event='1500m'))
    early_passes_to_front = laptimes[(laptimes['lap'] > 1) & 
                                     (laptimes['lap'] < 5) & 
                                     (laptimes['lap_start_position'] > 1) &
                                     (laptimes['lap_end_position'] == 1)]

    speed_up_sum = 0
    denominator = len(early_passes_to_front)

    for idx, early_pass in early_passes_to_front.iterrows():
        previous_laptime = laptimes[(laptimes['season'] == early_pass['season']) & 
                                    (laptimes['competition'] == early_pass['competition']) & 
                                    (laptimes['event'] == early_pass['event']) & 
                                    (laptimes['gender'] == early_pass['gender']) &
                                    (laptimes['round'] == early_pass['round']) &
                                    (laptimes['race'] == early_pass['race']) &
                                    (laptimes['instance_of_event_in_competition'] == early_pass['instance_of_event_in_competition']) &
                                    (laptimes['lap'] == early_pass['lap'] - 1)]
        if len(previous_laptime):
            speed_up_sum += previous_laptime.iloc[0]['laptime'] - early_pass['laptime']
        else:
            denominator -= 1

    return pn.indicators.Number(name='1500m Pace Instigation', 
                                value=round((speed_up_sum / denominator), 3), 
                                format='{value}s')

ui_template.sidebar.append(athlete_name)
ui_template.sidebar.append(event)
ui_template.sidebar.append(start_position)

ui_template.main.append(
    pn.Column(pn.Row(first_lap_positions, half_lap_500m, half_lap_500m_hist),
              pn.Row(start_performance_500m, fastest_leading_laptimes, fastest_following_laptimes),
              pn.Row(likely_lap_to_pass, x_plus_y_position_selection),
              pn.Row(leading_pace_1500m, pace_instigation_1500m))
)

ui_template.show()

Launching server at http://localhost:53079
